In [20]:
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas as hvplot
import matplotlib.pyplot as plt
import requests
import os
import csv
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import numpy as np

In [21]:
load_dotenv()

True

In [3]:
# Import API's/Data Sets 

In [22]:
alpaca_api_key = os.getenv("alpaca_api_key")
alpaca_secret_key = os.getenv("alpaca_secret_key")
print(type(alpaca_api_key))
print(type(alpaca_secret_key))
# Create the Alpaca API object
api = tradeapi.REST(
alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

<class 'str'>
<class 'str'>


In [23]:
snp_path = Path('SP_100_Symbol.csv')
tickers = []
with open(snp_path, 'r') as cvsfile: 
    snp_reader = csv.reader(cvsfile, delimiter= ',')
    header = next(snp_reader)   
    
    for column in snp_reader:
#         print(row)
        tickers.append(str(column[0]))
#         print(tickers)

ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


## Please select your stocks to build your portfolio

In [17]:
import tkinter
from tkinter import *

snp_tickers = ['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'ALL', 'AMGN', 'AMT', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BKNG', 'BLK', 'BMY', 'BRK.B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FB', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTX', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM']

window_snp = tkinter.Tk()
window_snp.title("S&P 100 Stocks")
label_snp = tkinter.Label(window_snp, text="Please choose up to 10 stocks from the following list").pack()
window_snp.geometry('700x400')


clicked = StringVar(window_snp)
clicked.set(snp_tickers[0]) # default value

menu = OptionMenu(window_snp, clicked, *snp_tickers)
menu.pack()

stocks_chosen = []
def stock_choice():
    stocks_chosen.append(clicked.get())
    print ("You've selected " + clicked.get())

button = Button(window_snp, text="OK", command=stock_choice)
button.pack()

mainloop()


You've selected AAPL
You've selected CVX
You've selected FB


In [24]:
stocks_chosen = list(dict.fromkeys(stocks_chosen))
print(stocks_chosen)

['AAPL', 'CVX', 'FB']


In [18]:
# Format current date as ISO format
start_date = pd.Timestamp("2020-08-14", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-08-15", tz="America/New_York").isoformat()

# Set the tickers
tickers = []
tickers.extend(stocks_chosen)

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
portfolio_df = api.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

portfolio_df_closing_prices = pd.DataFrame()

for ticker in tickers:
    portfolio_df_closing_prices[ticker] = portfolio_df[ticker]["close"]

portfolio_df_closing_prices.index = portfolio_df_closing_prices.index.date
portfolio_df_closing_prices.head()

,AAPL,CVX,FB
2020-08-14,459.78,90.36,261.24


In [25]:
start_date = pd.Timestamp("2020-08-14", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-08-15", tz="America/New_York").isoformat()

# Set the tickers
tickers = ['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'ALL', 'AMGN', 'AMT', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BKNG', 'BLK', 'BMY', 'BRK.B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FB', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTX', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM']

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
snp_df = api.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df


snp_df_closing_prices = pd.DataFrame()

for ticker in tickers:
    snp_df_closing_prices[ticker] = snp_df[ticker]["close"]

snp_df_closing_prices.index = snp_df_closing_prices.index.date
snp_df_closing_prices.head()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
2020-08-14,459.78,95.11,99.99,230.16,447.5,31,96.11,239.71,250.12,3148,...,323.86,191.92,160.74,37.78,196.63,58.79,41.905,25.3,132.54,43.21


In [38]:
## Analysis

In [26]:
# Format current date as ISO format
start_date = pd.Timestamp("2017-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2019-12-31", tz="America/New_York").isoformat()

# Set the tickers
tickers = []
tickers.extend(stocks_chosen)

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
portfolio_df = api.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

portfolio_df_closing_prices = pd.DataFrame()

for ticker in tickers:
    portfolio_df_closing_prices[ticker] = portfolio_df[ticker]["close"]

portfolio_df_closing_prices.index = portfolio_df_closing_prices.index.date
portfolio_df_closing_prices.head()

,AAPL,CVX,FB
2017-01-03,116.14,111.26,116.85
2017-01-04,116.02,117.81,118.70
2017-01-05,116.61,110.75,120.67
2017-01-06,117.91,110.31,123.43
2017-01-09,119.00,115.87,124.92


In [28]:
daily_returns = portfolio_df_closing_prices.pct_change().dropna()
daily_returns.head()

,AAPL,CVX,FB
2017-01-04,-0.001033,0.058871,0.015832
2017-01-05,0.005085,-0.059927,0.016596
2017-01-06,0.011148,-0.003973,0.022872
2017-01-09,0.009244,0.050403,0.012072
2017-01-10,0.000924,-0.063347,-0.004563


In [30]:
daily_std = daily_returns.std()
daily_std.head()

AAPL    0.015618
CVX     0.019436
FB      0.018299
dtype: float64

In [31]:
volatility = daily_std * np.sqrt(756)
volatility.sort_values(inplace=True)
volatility.head()

AAPL    0.429427
FB      0.503136
CVX     0.534393
dtype: float64

In [27]:
daily_cumulative_returns = (1+daily_returns).cumprod()
daily_cumulative_returns.head()

,AAPL,ALL,BAC,BMY
2017-01-04,0.998967,0.976406,1.018353,0.994451
2017-01-05,1.004047,0.971956,1.006214,1.011098
2017-01-06,1.015240,0.999865,1.007102,1.007987
2017-01-09,1.024625,0.985574,1.001296,1.004372
2017-01-10,1.025573,0.968316,1.018642,1.008744


In [34]:
#calculating holding period return
beg_prices =portfolio_df_closing_prices.iloc[0]
end_prices = portfolio_df_closing_prices.iloc[-1]

holding_prices = pd.concat([beg_prices, end_prices], axis = 1, join="outer")
holding_prices = holding_prices.transpose()

holding_period_return = holding_prices.pct_change().dropna()
holding_period_return

,AAPL,ALL,BAC,BMY
2019-12-31,1.526003,0.515572,0.562361,0.078525


In [36]:
#correlation 
correlation = daily_returns.corr()
correlation

,AAPL,ALL,BAC,BMY
AAPL,1.000000,0.271241,0.419895,0.302903
ALL,0.271241,1.000000,0.397705,0.160032
BAC,0.419895,0.397705,1.000000,0.221342
BMY,0.302903,0.160032,0.221342,1.000000


In [37]:
## Evaluating Reward-to-risk (Sharpe Ratio) of each stock
sharpe_ratios = (daily_returns.mean() * 756) / volatility
sharpe_ratios

AAPL    2.382812
ALL     1.355210
BAC     1.316792
BMY     0.398867
dtype: float64

In [ ]:
#monte_carlo

In [ ]:
threshold_window = tkinter.Tk()
threshold_window.title("Risk Assessment & Rebalancing Criteria")
threshold_label_1 = tkinter.Label(threshold_window, text="Please let us know how risk averse you are:")

risk_level = [
    "Low Risk",
    "Low-Medium Risk",
    "Medium Risk",
    "Medium-High Risk",
    "High Risk"
]

threshold_clicked = StringVar()
threshold_clicked.set(risk_level[0]) # default value
threshold_menu = OptionMenu(threshold_window, threshold_clicked, *risk_level)

threshold_label_1.grid(row=0, column=0)
threshold_menu.grid(row=0, column=1)

chosen_risk = []
def risk_choice():
    chosen_risk.append(threshold_clicked.get())
    print ("You've selected " + threshold_clicked.get())
    
threshold_button_1 = tkinter.Button(threshold_window, text="Select level of risk", command=risk_choice)
threshold_button_1.grid(row=1, column=0)

threshold_window.mainloop()

In [ ]:
risk_dict = {
    "Low Risk":{
        "Upper Threshold": 0.01,
        "Lower Threshold": 0.001,
    },
    "Low-Medium Risk":{
        "Upper Threshold": 0.1,
        "Lower Threshold": 0.01,
    },
    "Medium Risk":{
        "Upper Threshold": 0.3,
        "Lower Threshold": 0.1,
    },
    "Medium-High Risk":{
        "Upper Threshold": 0.5,
        "Lower Threshold": 0.3,
    },
    "High Risk":{
        "Upper Threshold": 0.7,
        "Lower Threshold": 0.5,
    },
}

In [ ]:
if chosen_risk == risk_dict[0]:
    print(f"We do not recommend the following {stock} because it's volatility is too high )
    print(f"However, we do recommend the following {stock} because it is highly correlated but with less volatility.")
elif chosen_risk == risk_dict[1]:
    print(f"We do not recommend the following {stock} because it's volatility is just a bit high based off of your preferences and our calculations.")
    print(f"However, we do recommend the following {stock} because it is highly correlated but with less volatility.")
elif chosen_risk == risk_dict[2]:
    print(f"We do not recommend the following {stock} because it's volatility is just a bit high based off of your preferences and our calculations.")
    print(f"However, we do recommend the following {stock} because it is highly correlated but with less volatility.")
elif chosen_risk == risk_dict[3]:
    print(f"We do not recommend the following {stock} because it's volatility is just a bit low based off of your preferences and our calculations.")
    print(f"However, we do recommend the following {stock} because it is highly correlated but with more volatility.")
elif chosen_risk == risk_dict[4]:
    print(f"We do not recommend the following {stock} because it's volatility is just a bit low based off of your preferences and our calculations.")
    print(f"However, we do recommend the following {stock} because it is highly correlated but with more volatility.")